## Prototype de Transaction et de Triggers SuperSmartMarket

* Recréation de la table vente à partir des logs
  * Préparation des données
  * Boucle sur les tickets pour insérer par "paquet"
  * Utilisation de la procédure InsertVente créé sur la base prototype

In [209]:
from dotenv import load_dotenv
from sqlalchemy import create_engine, Table, MetaData, select, text
from sqlalchemy.exc import SQLAlchemyError
import os
import pandas as pd
import polars as pl

In [210]:
df = pl.read_csv('./data/logs.csv')

In [211]:
df_update_prix = df.filter(pl.col('champs') == 'prix')

In [212]:
df_update = df.filter(
    pl.col('action') == 'UPDATE',
    pl.col('champs') != 'prix'
)

In [213]:
df_insert = df.filter(pl.col('action') == 'INSERT')

In [214]:
(
    df.group_by(
        pl.col('champs')
    )
    .agg(pl.len().alias('count'))
    .sort('count', descending=True)
)

champs,count
str,u32
"""CUSTUMER_ID""",41377
"""EAN""",41377
"""ID ticket""",41377
"""Date""",41377
"""id_employe""",41377
"""prix""",575
"""date_inscription""",20
"""hash_mdp""",7
null,2


In [215]:
df = (
    df.filter(
        (pl.col('action') == 'INSERT') &
        (pl.col('table_insert') == 'Ventes')
    )
    .select(
        pl.col(['id_ligne', 'champs', 'detail'])
    )
    .pivot(on='champs', index='id_ligne')
)

In [216]:
df = df.rename(mapping={'id_ligne':'id_bdd','CUSTUMER_ID':'customer_id','ID ticket':'id_ticket', 'Date':'date_achat', 'EAN':'ean'})

In [217]:
# Création de la liste des tickets
list_tickets = df.select(pl.col('id_ticket')).unique().to_series().to_list()

In [218]:
df = df.with_columns(
    pl.lit('2024-08-14').alias('date_achat')
)

In [219]:
print(df)

shape: (41_377, 6)
┌───────────────────┬──────────────────┬──────────────────┬───────────────┬────────────┬───────────┐
│ id_bdd            ┆ customer_id      ┆ id_employe       ┆ ean           ┆ date_achat ┆ id_ticket │
│ ---               ┆ ---              ┆ ---              ┆ ---           ┆ ---        ┆ ---       │
│ str               ┆ str              ┆ str              ┆ str           ┆ str        ┆ str       │
╞═══════════════════╪══════════════════╪══════════════════╪═══════════════╪════════════╪═══════════╡
│ 000MDBR8LMMFI8BWF ┆ CUST-F6TIGKY2H6W ┆ 4853b03deab973a1 ┆ 1987784879907 ┆ 2024-08-14 ┆ t_3839    │
│ EYR4KEE8MYGUP…    ┆ R                ┆ a8e466025bce5b…  ┆               ┆            ┆           │
│ 0031A86HBT87DS7FU ┆ CUST-B0T6XST4DGF ┆ afb0ad3a096ceea0 ┆ 6019185580672 ┆ 2024-08-14 ┆ t_4556    │
│ 1QBUN36UD67WE…    ┆ E                ┆ 9662ee8692cd57…  ┆               ┆            ┆           │
│ 003RNQDBFXN5XKH5H ┆ CUST-GHO8V34D1YB ┆ f6cd8ba3485769b3 ┆ 980638430995

_______

## Connection à la BDD et insertion

In [220]:
# Chargement des variables d'environnement et connexion à la BDD

load_dotenv()

# Configurer les informations à partir de .env
DB_SERVER = os.getenv('DB_SERVER')
DB_NAME = os.getenv('DB_NAME')
DB_USERNAME = os.getenv('DB_USERNAME')
DB_PASSWORD = os.getenv('DB_PASSWORD')

DRIVER ='mysql+pymysql'

connection_string = f"{DRIVER}://{DB_USERNAME}:{DB_PASSWORD}@{DB_SERVER}/{DB_NAME}"

engine = create_engine(connection_string)

In [221]:
# Passage de polars à pandas pour l'insertion via sqlalchemy
df = df.to_pandas()

In [222]:
with engine.connect() as conn: # Gestionnaire de contexte sqlalchemy
    
    # Création d'une boucle sur les tickets pour insérer les données en paquet dans une transaction
    for ticket in list_tickets:
        
        try:
            # Isolation des données pour le ticket en cours dans la boucle        
            df_transac = df[df["id_ticket"] == ticket]

            # Préparation des valeurs à insérer
            values = ", ".join([
                f"('{row['id_bdd']}', '{row['customer_id']}', '{row['id_employe']}', '{row['ean']}', '{row['date_achat']}', '{row['id_ticket']}')"
                for _, row in df_transac.iterrows()
            ])

            # Insertion des valeurs pour le ticket
            conn.execute(
                text("CALL InsertVente(:ticket_id, :vente_values)"),
                {
                    "ticket_id": ticket, 
                    "vente_values": values
                }
            )
            
            
        except SQLAlchemyError as e:
            pass
            # On ignore les erreurs de sqlalchemy pour continuer à insérer les données pour les autres tickets
        

In [223]:
# Génération d'une erreur pour tester le logging
## On insère des valeurs déjà présentent dans la BDD
values = """
('0EKV6VK8PCBWUEH2OGM10APVCJ5WHVTW6DD', 'CUST-21XPJKDCSBFY', 'f21c1650dd9340170266d47ec34eb6e8', '8998736834828', '2024-08-14', 't_2828'),
('0EKV6VK8PCBWUEH2OGM10APVCJ5WHVTW6XX', 'CUST-21XPJKDCSBFY', 'f21c1650dd9340170266d47ec34eb6e8', '8998736834828', '2024-08-14', 't_2828')
"""

ticket = "t_2828"

# Execute the procedure
try :
    with engine.connect() as conn:
        conn.execute(
            text("CALL InsertVente(:ticket_id, :vente_values)"),
            {"ticket_id": ticket, "vente_values": values}
        )
except SQLAlchemyError as e:

    print(f"SQLAlchemy Error: {str(e)}")

SQLAlchemy Error: (pymysql.err.IntegrityError) (1062, "Duplicate entry 't_2828-2024-12-12 16:13:33' for key 'transaction_log.PRIMARY'")
[SQL: CALL InsertVente(%(ticket_id)s, %(vente_values)s)]
[parameters: {'ticket_id': 't_2828', 'vente_values': "\n('0EKV6VK8PCBWUEH2OGM10APVCJ5WHVTW6DD', 'CUST-21XPJKDCSBFY', 'f21c1650dd9340170266d47ec34eb6e8', '8998736834828', '2024-08-14', 't_2828'),\n('0EKV6VK8PCBWUEH2OGM10APVCJ5WHVTW6XX', 'CUST-21XPJKDCSBFY', 'f21c1650dd9340170266d47ec34eb6e8', '8998736834828', '2024-08-14', 't_2828')\n"}]
(Background on this error at: https://sqlalche.me/e/20/gkpj)


In [224]:
# Réalisation d'un UPDATE du prix d'un des produits

try:
    with engine.connect() as conn:
        
        query = text("""
        UPDATE produit
        SET prix = 2.50
        WHERE ean = '8855635668141';
        """)
        
        result = conn.execute(query)
        conn.commit()
        print("Price updated successfully.")
        
except Exception as e:
    print(f"An error occurred: {e}")
    

Price updated successfully.


In [225]:
# Réalisation d'une erreur d'UPDATE du prix d'un des produits

try:
    with engine.connect() as conn:
        
        query = text("""
        UPDATE produit
        SET prix = '2024-08-15'
        WHERE ean = '8855635668141';
        """)
        
        result = conn.execute(query)
        conn.commit()
        print("Price updated successfully.")
        
except Exception as e:
    print(f"An error occurred: {e}")

An error occurred: (pymysql.err.DataError) (1366, "Incorrect decimal value: '2024-08-15' for column 'prix' at row 1")
[SQL: 
        UPDATE produit
        SET prix = '2024-08-15'
        WHERE ean = '8855635668141';
        ]
(Background on this error at: https://sqlalche.me/e/20/9h9h)
